<a href="https://colab.research.google.com/github/raafitt/RNN/blob/LSTM/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%2227_Streamlit_Sber_1d%22%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install requests schedule nest_asyncio streamlit altair

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Файл где создается и обучается модель

In [4]:
%%writefile model.py
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint,Callback
import streamlit as st

def create_model(input):
  n_features=1
  model = Sequential()
  model.add(LSTM(50, activation='relu', input_shape=input))
  model.add(Dense(10, activation='relu'))
  model.add(Dense(n_features))

  callbacks=ModelCheckpoint('best_model_1min.keras', mode='min',monitor='val_loss', save_best_only=True,verbose=True)

  class StreamlitCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
      st.write(f"Epoch {epoch+1}: loss = {logs['loss']}, val_loss = {logs['val_loss']}")

  model.compile(optimizer='adam', loss='mse')
  return model,callbacks,StreamlitCallback

def fit_model(model,train,val,callbacks):
  return model.fit(train, validation_data=val, epochs=5,callbacks=callbacks).history



Writing model.py


Вспомогательный файл, возвращающий прогноз

In [5]:
%%writefile lstm_model.py
import tensorflow as tf
import numpy as np
import pickle


model=tf.keras.models.load_model('/content/drive/MyDrive/27 streamlit/best_model_1d(2).keras')
# Загрузка scaler
with open('/content/drive/MyDrive/27 streamlit/scaler_1d.pkl', 'rb') as f:
  scaler = pickle.load(f)


# Функция расчета результата предсказания
def get_pred(x_test):
    n_input = 2
    x_test = np.reshape(x_test, (-1, n_input))
    x_test = np.reshape(x_test, (len(x_test) * n_input, 1))
    batch = len(x_test) // n_input
    x_test = scaler.transform(x_test)
    x_test = np.reshape(x_test, (batch, n_input, 1))
    # Вычисление и денормализация предсказания
    y_pred_unscaled = scaler.inverse_transform(model.predict(x_test, verbose=0))
    return np.round(y_pred_unscaled[0][0],2)

if __name__ == "__get_pred__":
    get_pred()

Writing lstm_model.py


# Файл с приложением (загрузка обучающих данных, обучение модели, демонстрация работы)

In [7]:
%%writefile main.py
import streamlit as st
import numpy as np
import io
import pandas as pd
from collections import deque
import re
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from io import StringIO
from model import create_model,fit_model
import altair as alt
import aiohttp
import asyncio
import json
import nest_asyncio
from lstm_model import get_pred



st.title('Загрузка данных, обучение и инференс модели')
uploaded_file = st.file_uploader("Загрузите датасет в формате csv")
if uploaded_file is not None:
    price = pd.read_csv(uploaded_file,sep=';', index_col='time', usecols = ['time','c'], parse_dates=['time'])
    price.dropna(inplace=True)
    price.replace({',':'.'},regex=True,inplace=True)
    price = price.sort_index()
    st.write(price.head())

    train_data = price[:'2024-09-30']
    test_data = price['2024-09-30':]

    scaler = MinMaxScaler()
    scaler.fit(train_data)
    scaled_train_data = scaler.transform(train_data)
    scaled_test_data = scaler.transform(test_data)

    n_input = 2  # Размерность входных данных
    n_features = 1 # Размерность выходных данных
    BATCH_SIZE = 1 # Размер пакета

    generator = TimeseriesGenerator(scaled_train_data, scaled_train_data, length=n_input, batch_size=BATCH_SIZE)
    st.write(f'Форма обучающего пакета: {generator[0][0].shape}, y: {generator[0][1].shape}')

    validator = TimeseriesGenerator(scaled_test_data, scaled_test_data, length=n_input, batch_size=BATCH_SIZE)
    st.write(f'Форма валидационного пакета: {validator[0][0].shape}, y: {validator[0][1].shape}')

    tester = TimeseriesGenerator(scaled_test_data, scaled_test_data, length=n_input, batch_size=scaled_test_data.shape[0])
    x_test, y_test = tester[0]
    st.write(f'Форма тестовой выборки: {x_test.shape}, y: {y_test.shape}')

    model,callb1,callb2=create_model(generator[0][0].shape[1:])
    if st.button('Показать архитектуру модели'):
      st.write('Архитектура модели')
      # Перенаправление вывода summary в строку
      summary_str = io.StringIO()
      model.summary(print_fn=lambda x: summary_str.write(x + '\n'))
      st.text(summary_str.getvalue())

    st.empty()
    if st.button('Начать обучение'):
      st.write('Начало обучения...')
      st.write(fit_model(model,generator,validator,[callb1,callb2()]))
      st.write('Обучение завершено.')

    #Демонстрация работы модели(предсказыване цен на акции сбера в реальном времени)
    st.empty()
    if st.button('Запустить прогноз цен акций Сбера'):
      nest_asyncio.apply()
      #limit = n_input
      #interval = '1min'

      url='https://api-invest-gw.tinkoff.ru/market-data-history/api/public/v1/candles?instrument_id=e6123145-9665-43e0-8413-cd61b8aa9b13&limit=600&interval=1min&appName=invest_terminal&appVersion=2.0.0&sessionId=TCPiOzihJNgYQgKmUaO0y3hIqOiBdDCF.ds-prod-api-119'
      #Функцмя извлечения цены закрытия
      def get_close(candles):
          closes = []
          for elem in candles:
              closes.append(elem['c'])
          return closes

      #Асинхронная Функция получения котировок
      async def fetch_quotes(session, url):
          async with session.get(url) as response:
              return await response.json()

      async def get_quotes_periodically(url, interval):
          async with aiohttp.ClientSession() as session:
              #Для ограничения точек на графике используется очередь
              data = deque(maxlen=7)
              chart1 = st.empty()
              chart2 = st.empty()
              # Основной цикл с периодическим обновлением данных
              while True:
                  try:
                      quotes = await fetch_quotes(session, url)
                      candles = quotes['payload']['candles']
                      closes = get_close(list(candles))

                      # Получение предсказания
                      pred_value = np.round(get_pred(closes), 2)

                      # Извлекаем минуты и секунды
                      time = pd.Timestamp.now()
                      # Добавляем 60 секунд к текущему времени
                      next_time = time + pd.Timedelta(seconds=60)

                      data.append({'next_time': next_time,'time': time, 'pred_price': pred_value, 'price': closes[1]})
                      # Создаем DataFrame для графика
                      df = pd.DataFrame(data)

                      y_min_pred = df['pred_price'].min()-df['pred_price'].min()*0.0001
                      y_max_pred = df['pred_price'].max()+df['pred_price'].max()*0.0001
                      y_min = df['price'].min()-df['price'].min()*0.0001
                      y_max = df['price'].max()+df['price'].max()*0.0001

                      # Построение двух линий: одна для реальных цен, другая для предсказанных
                      price_line = alt.Chart(df).mark_line(color='blue').encode(
                            x=alt.X('time:T', axis=alt.Axis(format='%M', title='Минуты')),
                            y=alt.Y('price:Q', scale=alt.Scale(domain=[y_min, y_max]))
                      ).properties(title='График изменения цены',width=666, height=400)

                      # Добавление точек на линию
                      price_points = alt.Chart(df).mark_point(color='blue').encode(
                          x='time:T',
                          y='price:Q'
                      )

                      # Добавление текста над точками
                      price_text = alt.Chart(df).mark_text(align='left', dx=5, dy=-5).encode(
                          x='time:T',
                          y='price:Q',
                          text='price:Q'
                      )

                      # Объединение всех частей графика
                      final_chart = price_line + price_points + price_text

                      pred_price_line = alt.Chart(df).mark_line(color='red').encode(
                            x=alt.X('next_time:T', axis=alt.Axis(format='%M', title='Минуты')),
                            y=alt.Y('pred_price:Q', scale=alt.Scale(domain=[y_min_pred, y_max_pred]))
                      ).properties(title='График изменения цены предсказания',width=666,height=400)

                      # Добавление точек на линию
                      pred_price_points = alt.Chart(df).mark_point(color='red').encode(
                          x='next_time:T',
                          y='pred_price:Q'
                      )

                      # Добавление текста над точками
                      pred_price_text = alt.Chart(df).mark_text(align='left', dx=5, dy=-5).encode(
                          x='next_time:T',
                          y='pred_price:Q',
                          text='pred_price:Q'
                      )

                      # Объединение всех частей графика
                      pred_final_chart = pred_price_line + pred_price_points + pred_price_text

                      # Отображение графика с центрированием


                      chart1.altair_chart(pred_final_chart)
                      # Задержка в 1 секунду
                      await asyncio.sleep(1)
                      chart2.altair_chart(final_chart)






                  except Exception as e:
                      st.error(f"Ошибка при получении котировок: {e}")

                  await asyncio.sleep(interval)

      # Запуск асинхронной функции
      interval = 60  # Интервал в секундах
      asyncio.run(get_quotes_periodically(url, interval))

Overwriting main.py


In [10]:
!streamlit run main.py --server.address=localhost >/content/logs.txt & ssh -o "StrictHostKeyChecking no" -R 80:localhost:8501 serveo.net

ssh: connect to host serveo.net port 22: Connection refused


In [9]:
!streamlit run main.py --server.address=localhost >/content/logs.txt & ssh -o "StrictHostKeyChecking no" -R 80:localhost:8501 nokey@localhost.run


Welcome to localhost.run!

Follow your favourite reverse tunnel at [https://twitter.com/localhost_run].

To set up and manage custom domains go to https://admin.localhost.run/

More details on custom domains (and how to enable subdomains of your custom
domain) at https://localhost.run/docs/custom-domains

If you get a permission denied error check the faq for how to connect with a key or
create a free tunnel without a key at [http://localhost:3000/docs/faq#generating-an-ssh-key].

To explore using localhost.run visit the documentation site:
https://localhost.run/docs/


** your connection id is 3519b7ca-145f-47fb-9cc5-e3e85d2cb03a, please mention it if you send me a message about an issue. **


********************************************************************************
Free plans are temporarily disabled. Please check back in a few days.

"Why?" you ask.

Over the past few days there has been an immense spike in free plan usage,
and along with this bills suddently increased to 15